In [1]:
import numpy as np
import os.path as osp
import pickle, json, random

In [2]:
import os, math
import os.path as osp
from copy import deepcopy
from functools import partial
from pprint import pprint

In [3]:
import sacred
import torch
import torch.nn as nn
import torch.nn.functional as F
from sacred import SETTINGS
from sacred.utils import apply_backspaces_and_linefeeds
from torch.backends import cudnn
from torch.optim import SGD, Adam, AdamW, lr_scheduler
from tqdm import tqdm

from torch.backends import cudnn
# from visdom_logger import VisdomLogger

In [4]:
from torch.utils.data import DataLoader, RandomSampler, BatchSampler
from typing import NamedTuple, Optional, List

In [5]:
from models.matcher import MatchERT
from models.ingredient import model_ingredient, get_model

In [6]:
from models.ingredient import model_ingredient, get_model
from utils import state_dict_to_cpu, num_of_trainable_params
from utils import pickle_load, pickle_save
#from utils.data.utils import TripletSampler
from utils import BinaryCrossEntropyWithLogits
from utils.data.dataset_ingredient import data_ingredient, get_loaders
from utils.training import train_one_epoch, evaluate_viquae

In [7]:
from utils import pickle_load
from sacred import Experiment
from utils.data.dataset_ingredient import data_ingredient, get_loaders
from utils.data.dataset import FeatureDataset

In [8]:
ex = sacred.Experiment('RRT Training', ingredients=[data_ingredient, model_ingredient], interactive=True)
# Filter backspaces and linefeeds
SETTINGS.CAPTURE_MODE = 'sys'
ex.captured_out_filter = apply_backspaces_and_linefeeds

In [9]:
epochs = 15
lr = 0.0001
momentum = 0.
nesterov = False
weight_decay = 5e-4
optim = 'adamw'
scheduler = 'multistep'
max_norm = 0.0
seed = 0

visdom_port = None
visdom_freq = 100
cpu = False  # Force training on CPU
cudnn_flag = 'benchmark'
temp_dir = osp.join('outputs', 'temp')

no_bias_decay = False
loss = 'bce'
scheduler_tau = [16, 18]
scheduler_gamma = 0.1

resume = '/mnt/beegfs/home/smessoud/RerankingTransformer/RRT_GLD/rrt_gld_ckpts/r50_gldv1.pt'

In [10]:
epochs, cpu, cudnn_flag, visdom_port, visdom_freq, temp_dir, seed, no_bias_decay, max_norm, resume

(15,
 False,
 'benchmark',
 None,
 100,
 'outputs/temp',
 0,
 False,
 0.0,
 '/mnt/beegfs/home/smessoud/RerankingTransformer/RRT_GLD/rrt_gld_ckpts/r50_gldv1.pt')

In [11]:
def get_optimizer_scheduler(parameters, optim, loader_length, epochs, lr, momentum, nesterov, weight_decay, scheduler, scheduler_tau, scheduler_gamma, lr_step=None):
    if optim == 'sgd':
        optimizer = SGD(parameters, lr=lr, momentum=momentum, weight_decay=weight_decay, nesterov=True if nesterov and momentum else False)
    elif optim == 'adam':
        optimizer = Adam(parameters, lr=lr, weight_decay=weight_decay) 
    else:
        optimizer = AdamW(parameters, lr=lr, weight_decay=weight_decay)
    
    if epochs == 0:
        scheduler = None
        update_per_iteration = None
    elif scheduler == 'cos':
        # scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs * loader_length, eta_min=0.000005)
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=0.000001)
        update_per_iteration = False
    elif scheduler == 'warmcos':
        # warm_cosine = lambda i: min((i + 1) / 3, (1 + math.cos(math.pi * i / (epochs * loader_length))) / 2)
        warm_cosine = lambda i: min((i + 1) / 3, (1 + math.cos(math.pi * i / epochs)) / 2)
        scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=warm_cosine)
        update_per_iteration = False
    elif scheduler == 'multistep':
        scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=scheduler_tau, gamma=scheduler_gamma)
        update_per_iteration = False
    elif scheduler == 'warmstep':
        warm_step = lambda i: min((i + 1) / 100, 1) * 0.1 ** (i // (lr_step * loader_length))
        scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=warm_step)
        update_per_iteration = True
    else:
        scheduler = lr_scheduler.StepLR(optimizer, epochs * loader_length)
        update_per_iteration = True

    return optimizer, (scheduler, update_per_iteration)



In [12]:
def get_loss(loss):
    if loss == 'bce':
        return BinaryCrossEntropyWithLogits()
    else:
        raise Exception('Unsupported loss {}'.format(loss))

In [13]:
device = torch.device('cuda:0' if torch.cuda.is_available() and not cpu else 'cpu')
# callback = VisdomLogger(port=visdom_port) if visdom_port else None
if cudnn_flag == 'deterministic':
    setattr(cudnn, cudnn_flag, True)

In [14]:
def read_file(filename):
    with open(filename) as f:
        lines = f.read().splitlines()
    return lines

In [15]:
name = 'tuto_viquae_tuto_r50_gldv1'
set_name = 'tuto'
train_txt = ('tuto_query.txt', 'tuto_gallery.txt')
test_txt = ('tuto_query.txt', 'tuto_selection.txt')
train_data_dir = 'data/viquae_for_rrt'
test_data_dir  = 'data/viquae_for_rrt'
test_gnd_file = 'gnd_tuto.pkl'
train_gnd_file = 'training_gnd_'+set_name+'.pkl'
#train_gnd_file = 'gnd_'+set_name+'.pkl'
desc_name = 'r50_gldv1'
sampler = 'triplet'
split_char  = ';;'

In [16]:
"""
name = 'train_viquae_dev_r50_gldv1'
set_name = 'train'
train_txt = (set_name+'_query.txt', set_name+'_selection.txt')
test_txt = ('dev_query.txt', 'dev_selection.txt')
train_data_dir = 'data/viquae_for_rrt'
test_data_dir  = 'data/viquae_for_rrt'
train_gnd_file = 'gnd_train.pkl'
test_gnd_file = 'gnd_dev.pkl'
desc_name = 'r50_gldv1'
sampler = 'triplet'
split_char  = ';;'
"""

"\nname = 'train_viquae_dev_r50_gldv1'\nset_name = 'train'\ntrain_txt = (set_name+'_query.txt', set_name+'_selection.txt')\ntest_txt = ('dev_query.txt', 'dev_selection.txt')\ntrain_data_dir = 'data/viquae_for_rrt'\ntest_data_dir  = 'data/viquae_for_rrt'\ntrain_gnd_file = 'gnd_train.pkl'\ntest_gnd_file = 'gnd_dev.pkl'\ndesc_name = 'r50_gldv1'\nsampler = 'triplet'\nsplit_char  = ';;'\n"

In [17]:
len(train_txt)

2

In [18]:
def get_sets(desc_name, 
        train_data_dir, test_data_dir, 
        train_txt, test_txt, train_gnd_file,
        test_gnd_file, max_sequence_len,
        split_char):
    ####################################################################################################################################
    train_gnd_data  = None if train_gnd_file is None else pickle_load(osp.join(train_data_dir, train_gnd_file))
    train_lines     = read_file(osp.join(train_data_dir, train_txt[1]))
    train_q_lines   = read_file(osp.join(train_data_dir, train_txt[0]))
    train_samples   = [(line.split(split_char)[0], int(line.split(split_char)[1]), int(line.split(split_char)[2]), int(line.split(split_char)[3])) for line in train_lines]
    train_q_samples = [(line.split(split_char)[0], int(line.split(split_char)[1]), int(line.split(split_char)[2]), int(line.split(split_char)[3])) for line in train_q_lines]
    train_set       = FeatureDataset(train_data_dir, train_samples,   desc_name, max_sequence_len, gnd_data=train_gnd_data)
    query_train_set = FeatureDataset(train_data_dir, train_q_samples, desc_name, max_sequence_len, gnd_data=train_gnd_data)
    ####################################################################################################################################
    test_gnd_data = None if test_gnd_file is None else pickle_load(osp.join(test_data_dir, test_gnd_file))
    query_lines   = read_file(osp.join(test_data_dir, test_txt[0]))
    gallery_lines = read_file(osp.join(test_data_dir, test_txt[1]))
    query_samples   = [(line.split(split_char)[0], int(line.split(split_char)[1]), int(line.split(split_char)[2]), int(line.split(split_char)[3])) for line in query_lines]
    gallery_samples = [(line.split(split_char)[0], int(line.split(split_char)[1]), int(line.split(split_char)[2]), int(line.split(split_char)[3])) for line in gallery_lines]
    gallery_set = FeatureDataset(test_data_dir, gallery_samples, desc_name, max_sequence_len)
    query_set   = FeatureDataset(test_data_dir, query_samples,   desc_name, max_sequence_len, gnd_data=test_gnd_data)
        
    return (train_set, query_train_set), (query_set, gallery_set)


In [19]:
def get_loaders(desc_name, train_data_dir, 
    batch_size, test_batch_size, 
    num_workers, pin_memory, 
    sampler, recalls, set_name,
    num_candidates=100,):

    (train_set, query_train_set), (query_set, gallery_set) = get_sets(
        desc_name=desc_name, train_data_dir=train_data_dir, 
        test_data_dir=train_data_dir, train_txt=train_txt, 
        test_txt=test_txt, train_gnd_file=train_gnd_file, 
        test_gnd_file=test_gnd_file, 
        max_sequence_len=max_sequence_len, 
        split_char=split_char)

    if sampler == 'random':
        train_sampler = BatchSampler(RandomSampler(train_set), batch_size=batch_size, drop_last=False)
    elif sampler == 'triplet':
        #s_name = set_name
        #if s_name != '':
        #    s_name = set_name + '_'
        #def map_nnids_labels(train_data_dir, train_gnd_file, s_categories):
        #    gnd =  pickle_load(osp.join(train_data_dir, train_gnd_file))
        #    selection_gallery = gnd['simlist']
        #    s_categories = s_categories.reshape(np.array(selection_gallery).shape)
        #    selection_ids_to_cat_dict = [{k: s_categories[i][k] for k in range(len(selection_gallery[i]))} for i in range(len(selection_gallery))]
        #    return selection_ids_to_cat_dict
        #s_path = train_data_dir+'/'+set_name+'_s_categories.txt'
        #print('s_path: ', s_path)
        #s_categories = np.loadtxt(s_path, dtype='int64')
        #print('s_categories: ', s_categories.shape)
        #map_nnids_labels = map_nnids_labels(train_data_dir, train_gnd_file, s_categories)
        train_nn_inds = osp.join(train_data_dir, set_name + '_nn_inds_%s.pkl'%desc_name)
        gnd_data = train_set.gnd_data['gnd']
        train_sampler = TripletSampler(query_train_set.targets, batch_size, train_nn_inds, num_candidates, gnd_data)
    else:
        raise ValueError('Invalid choice of sampler ({}).'.format(sampler))
    train_loader = DataLoader(train_set, batch_sampler=train_sampler, num_workers=num_workers, pin_memory=pin_memory)
    query_train_loader = DataLoader(query_train_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)
        
    query_loader   = DataLoader(query_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)
    gallery_loader = DataLoader(gallery_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)

    return MetricLoaders(train=train_loader, query_train=query_train_loader, query=query_loader, gallery=gallery_loader, num_classes=len(train_set.categories),set_name=set_name), recalls


In [20]:
train_data_dir = '/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt'

In [21]:
class MetricLoaders(NamedTuple):
    train: DataLoader
    num_classes: int
    query: DataLoader
    query_train: DataLoader
    set_name: str = ''
    gallery: Optional[DataLoader] = None

In [22]:
batch_size      = 32
test_batch_size = 32
max_sequence_len = 500
num_workers = 8  # number of workers used ot load the data
pin_memory  = True  # use the pin_memory option of DataLoader 
num_candidates = 100
recalls = [1, 5, 6, 10]

In [23]:
class TripletSampler():
    def __init__(self, labels, batch_size, nn_inds_path, num_candidates, gnd_data, min_pos=5):
        self.batch_size     = batch_size
        self.num_candidates = num_candidates
        self.cache_nn_inds  = pickle_load(nn_inds_path)
        self.labels = labels
        self.gnd_data = gnd_data
        print('nn_inds_path: ', nn_inds_path)
        print('labels len: ', len(labels))
        assert (len(self.cache_nn_inds) == len(labels))
        #############################################################################
        ## Collect valid tuples
        valids = np.zeros_like(labels)
        for i in range(len(self.cache_nn_inds)):
            #nnids = self.cache_nn_inds[i]
            #query_label = labels[i]
            #index_labels = np.array([map_nnids_labels[i][j] for j in nnids])
            #index_labels = np.array([labels[j] for j in nnids])
            #positives = np.where(index_labels == query_label)[0]
            positives = self.gnd_data[i]['r_easy']
            if len(positives) < min_pos:
                continue
            valids[i] = 1
        self.valids = np.where(valids > 0)[0]
        self.num_samples = len(self.valids)

    def __iter__(self):
        batch = []
        cands = torch.randperm(self.num_samples).tolist()
        for i in range(len(cands)):
            query_idx = self.valids[cands[i]]
            anchor_idx = self.gnd_data[query_idx]['anchor_idx']
            #nnids = self.cache_nn_inds[anchor_idx]

            positive_inds = self.gnd_data[query_idx]['g_easy']
            negative_inds = self.gnd_data[query_idx]['g_junk']
            assert(len(positive_inds) > 0)
            assert(len(negative_inds) > 0)

            random.shuffle(positive_inds)
            random.shuffle(negative_inds)

            batch.append(anchor_idx)
            batch.append(positive_inds[0]) 
            batch.append(negative_inds[0])

            if len(batch) >= self.batch_size:
                yield batch
                batch = []
                
        if len(batch) > 0:
            yield batch

    def __len__(self):
        return (self.num_samples * 3 + self.batch_size - 1) // self.batch_size



In [24]:
nn_inds_path = '/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt/train_nn_inds_r50_gldv1.pkl'
cache_nn_inds  = pickle_load(nn_inds_path)
cache_nn_inds.shape

(1190, 100)

In [25]:
torch.manual_seed(seed)
loaders, recall_ks = get_loaders('r50_gldv1', train_data_dir, 
    batch_size, test_batch_size, 
    num_workers, pin_memory, 
    sampler, recalls, set_name,
    num_candidates=100)

nn_inds_path:  /mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt/tuto_nn_inds_r50_gldv1.pkl
labels len:  100


In [26]:
def get_model(num_global_features, num_local_features, seq_len, dim_K, dim_feedforward, nhead, num_encoder_layers, dropout, activation, normalize_before):
    return MatchERT(d_global=num_global_features, d_model=num_local_features, seq_len=seq_len, d_K=dim_K, nhead=nhead, num_encoder_layers=num_encoder_layers, 
            dim_feedforward=dim_feedforward, dropout=dropout, activation=activation, normalize_before=normalize_before)

In [27]:
name = 'rrt'
num_global_features = 2048  
num_local_features = 128  
seq_len = 1004
dim_K = 256
dim_feedforward = 1024
nhead = 4
num_encoder_layers = 6
dropout = 0.0 
activation = "relu"
normalize_before = False

In [28]:
torch.manual_seed(seed+1)
model = get_model(num_global_features,num_local_features,seq_len,dim_K,dim_feedforward,nhead,num_encoder_layers,dropout,activation,normalize_before)

In [29]:
if resume is not None:
    checkpoint = torch.load(resume, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['state'], strict=True)
print('# of trainable parameters: ', num_of_trainable_params(model))
class_loss = get_loss(loss)
nn_inds_path = osp.join(loaders.query.dataset.data_dir, loaders.set_name + '_nn_inds_%s.pkl'%loaders.query.dataset.desc_name)
cache_nn_inds = torch.from_numpy(pickle_load(nn_inds_path)).long()

# of trainable parameters:  2243201


In [30]:
torch.manual_seed(seed+2)
model.to(device)
model = nn.DataParallel(model)
parameters = []
if no_bias_decay:
    parameters.append({'params': [par for par in model.parameters() if par.dim() != 1]})
    parameters.append({'params': [par for par in model.parameters() if par.dim() == 1], 'weight_decay': 0})
else:
    parameters.append({'params': model.parameters()})
optimizer, scheduler = get_optimizer_scheduler(parameters=parameters, loader_length=len(loaders.train),
                                               optim=optim, epochs=epochs, lr=lr,
                                               momentum=momentum, nesterov=nesterov, weight_decay=weight_decay,
                                               scheduler=scheduler, scheduler_tau=scheduler_tau, 
                                               scheduler_gamma=scheduler_gamma, lr_step=None)
if resume is not None and checkpoint.get('optim', None) is not None:
    optimizer.load_state_dict(checkpoint['optim'])
    del checkpoint


In [31]:
from utils.metrics import *
def evaluate_viquae(
        model: nn.Module,
        cache_nn_inds: torch.Tensor,
        query_loader: DataLoader,
        gallery_loader: DataLoader,
        recall: List[int]):
    model.eval()
    device = next(model.parameters()).device
    to_device = lambda x: x.to(device, non_blocking=True)

    query_global, query_local, query_mask, query_scales, query_positions, query_names = [], [], [], [], [], []
    gallery_global, gallery_local, gallery_mask, gallery_scales, gallery_positions, gallery_names = [], [], [], [], [], []

    with torch.no_grad():
        for entry in tqdm(query_loader, desc='Extracting query features', leave=False, ncols=80):
            q_global, q_local, q_mask, q_scales, q_positions, _, q_names = entry
            query_global.append(q_global.cpu())
            query_local.append(q_local.cpu())
            query_mask.append(q_mask.cpu())
            query_scales.append(q_scales.cpu())
            query_positions.append(q_positions.cpu())
            query_names.extend(list(q_names))
            torch.cuda.empty_cache()

        query_global    = torch.cat(query_global, 0)
        query_local     = torch.cat(query_local, 0)
        query_mask      = torch.cat(query_mask, 0)
        query_scales    = torch.cat(query_scales, 0)
        query_positions = torch.cat(query_positions, 0)

        for entry in tqdm(gallery_loader, desc='Extracting gallery features', leave=False, ncols=80):
            g_global, g_local, g_mask, g_scales, g_positions, _, g_names = entry
            gallery_global.append(g_global.cpu())
            gallery_local.append(g_local.cpu())
            gallery_mask.append(g_mask.cpu())
            gallery_scales.append(g_scales.cpu())
            gallery_positions.append(g_positions.cpu())
            gallery_names.extend(list(g_names))
            torch.cuda.empty_cache()
            
        gallery_global    = torch.cat(gallery_global, 0)
        gallery_local     = torch.cat(gallery_local, 0)
        gallery_mask      = torch.cat(gallery_mask, 0)
        gallery_scales    = torch.cat(gallery_scales, 0)
        gallery_positions = torch.cat(gallery_positions, 0)

        torch.cuda.empty_cache()
        evaluate_function = partial(mean_average_precision_viquae_rerank, model=model, cache_nn_inds=cache_nn_inds,
            query_global=query_global, query_local=query_local, query_mask=query_mask, query_scales=query_scales, query_positions=query_positions, 
            gallery_global=gallery_global, gallery_local=gallery_local, gallery_mask=gallery_mask, gallery_scales=gallery_scales, gallery_positions=gallery_positions, 
            ks=recall, 
            gnd=query_loader.dataset.gnd_data,
        )
        metrics = evaluate_function()
    return metrics 



In [44]:
def read_entry_once(
    query_loader: DataLoader,
    gallery_loader: DataLoader):
    
    query_global, query_local, query_mask, query_scales, query_positions, query_names = [], [], [], [], [], []
    gallery_global, gallery_local, gallery_mask, gallery_scales, gallery_positions, gallery_names = [], [], [], [], [], []

    print("READING ENTRIES FOR THE FIRST TIME")
    
    with torch.no_grad():
        for entry in tqdm(query_loader, desc='Extracting query features', leave=False, ncols=80):
            q_global, q_local, q_mask, q_scales, q_positions, _, q_names = entry
            query_global.append(q_global.cpu())
            query_local.append(q_local.cpu())
            query_mask.append(q_mask.cpu())
            query_scales.append(q_scales.cpu())
            query_positions.append(q_positions.cpu())
            query_names.extend(list(q_names))
            torch.cuda.empty_cache()

        query_global    = torch.cat(query_global, 0)
        query_local     = torch.cat(query_local, 0)
        query_mask      = torch.cat(query_mask, 0)
        query_scales    = torch.cat(query_scales, 0)
        query_positions = torch.cat(query_positions, 0)

        for entry in tqdm(gallery_loader, desc='Extracting gallery features', leave=False, ncols=80):
            g_global, g_local, g_mask, g_scales, g_positions, _, g_names = entry
            gallery_global.append(g_global.cpu())
            gallery_local.append(g_local.cpu())
            gallery_mask.append(g_mask.cpu())
            gallery_scales.append(g_scales.cpu())
            gallery_positions.append(g_positions.cpu())
            gallery_names.extend(list(g_names))
            torch.cuda.empty_cache()

        gallery_global    = torch.cat(gallery_global, 0)
        gallery_local     = torch.cat(gallery_local, 0)
        gallery_mask      = torch.cat(gallery_mask, 0)
        gallery_scales    = torch.cat(gallery_scales, 0)
        gallery_positions = torch.cat(gallery_positions, 0)
    
    query_feats   = [query_global, query_local, query_mask, query_scales, query_positions, query_names]
    gallery_feats = [gallery_global, gallery_local, gallery_mask, gallery_scales, gallery_positions, gallery_names]
    
    return query_feats, gallery_feats

In [50]:
from utils.metrics import mean_average_precision_viquae_rerank

def fast_evaluate_viquae(
    model: nn.Module,
    cache_nn_inds: torch.Tensor,
    query_loader: DataLoader,
    gallery_loader: DataLoader,
    recall: List[int],
    query_feats, 
    gallery_feats):
    
    model.eval()
    device = next(model.parameters()).device
    to_device = lambda x: x.to(device, non_blocking=True)
    
    if len(query_feats) == 0:
        query_feats, gallery_feats = read_entry_once(query_loader, gallery_loader)
    
    query_global, query_local, query_mask, query_scales, query_positions, query_names = query_feats
    gallery_global, gallery_local, gallery_mask, gallery_scales, gallery_positions, gallery_names = gallery_feats
    
    torch.cuda.empty_cache()
    
    fast_evaluate_function = partial(mean_average_precision_viquae_rerank, model=model, cache_nn_inds=cache_nn_inds,
        query_global=query_global, query_local=query_local, query_mask=query_mask, query_scales=query_scales, query_positions=query_positions, 
        gallery_global=gallery_global, gallery_local=gallery_local, gallery_mask=gallery_mask, gallery_scales=gallery_scales, gallery_positions=gallery_positions, 
        ks=recall, 
        gnd=query_loader.dataset.gnd_data,
    )
    metrics = fast_evaluate_function()
    
    return metrics, query_feats, gallery_feats

In [46]:
torch.manual_seed(seed+3)
# setup partial function to simplify call
query_feats, gallery_feats = [], []

eval_function = partial(fast_evaluate_viquae, model=model, 
    cache_nn_inds=cache_nn_inds,
    recall=recall_ks, query_loader=loaders.query, gallery_loader=loaders.gallery,
    query_feats=query_feats, gallery_feats=gallery_feats)

In [47]:
result = eval_function()
pprint(result)
best_val = (0, result, deepcopy(model.state_dict()))

Extracting query features:   0%|                          | 0/4 [00:00<?, ?it/s]

READING ENTRIES FOR THE FIRST TIME


100%|██████████| 100/100 [01:02<00:00,  1.59it/s]                               


{'map': 30.3, 'mrr': 43.01, 'precision': 12.7, 'hit_rate': 83.0, 'recall': 78.87, 'map@1': 10.48, 'mrr@1': 32.0, 'precision@1': 32.0, 'hit_rate@1': 32.0, 'recall@1': 10.48, 'map@5': 18.01, 'mrr@5': 40.73, 'precision@5': 19.8, 'hit_rate@5': 55.0, 'recall@5': 24.23, 'map@6': 19.09, 'mrr@6': 41.23, 'precision@6': 19.33, 'hit_rate@6': 58.0, 'recall@6': 27.16, 'map@10': 22.07, 'mrr@10': 42.13, 'precision@10': 17.4, 'hit_rate@10': 65.0, 'recall@10': 35.78}
({'hit_rate': 83.0,
  'hit_rate@1': 32.0,
  'hit_rate@10': 65.0,
  'hit_rate@5': 55.0,
  'hit_rate@6': 58.0,
  'map': 30.3,
  'map@1': 10.48,
  'map@10': 22.07,
  'map@5': 18.01,
  'map@6': 19.09,
  'mrr': 43.01,
  'mrr@1': 32.0,
  'mrr@10': 42.13,
  'mrr@5': 40.73,
  'mrr@6': 41.23,
  'precision': 12.7,
  'precision@1': 32.0,
  'precision@10': 17.4,
  'precision@5': 19.8,
  'precision@6': 19.33,
  'recall': 78.87,
  'recall@1': 10.48,
  'recall@10': 35.78,
  'recall@5': 24.23,
  'recall@6': 27.16},
 [tensor([[-0.0106, -0.0303, -0.0169,  .

   '512px-Bryce_Dessner_2015',
   '512px-Tableau_farhat',
   '512px-Leoš_Janáček',
   '512px-Jonathan_Karp',
   '512px-Manja_Behrens_am_4.10.1954_beim_Matinee_in_der_Volksbühne_Ost-Berlin_(cropped)',
   '512px-OnSet',
   '512px-Michael_Nyman_Sant_Cugat',
   '512px-RezistantaRomanaAntiCommunista-Dumitru-Carlaont',
   '512px-Philip_Glass_1',
   '512px-Teresa_Palmer,_2012',
   '512px-René-Louis_Baron-Pierre_Douglas_et_Alain_Souchon',
   '512px-Marc_Blitzstein_1943',
   '512px-Wilhelm_Meyer-Förster_-_Stadsschouwburg_1904-1905_(2)',
   '512px-Peterborough_Town_House',
   '512px-ChristopherChaplin',
   "512px-Queen's_Theatre_London_2011_1",
   '512px-Reşit_Süreyya_Gürsey',
   '512px-Подлас_Кузьма_Петрович',
   '512px-GoldenGateBridge-001',
   '512px-New_Orleans_skyline-02',
   '512px-LightningVolt_Mackinac_Bridge',
   '512px-CrescentCityConnection',
   '512px-Haleboggsbridge',
   '512px-NewYorkStateThruway(I-87)',
   '512px-Lake_Pontchartrain_Causeway_south',
   '512px-Philadelphia_cityscape

   '512px-Gymnastics_brokenchopstick',
   '512px-Matthew_Webb',
   '512px-Muttlebury_SD_Vanity_Fair_1890-03-22',
   '512px-Alfred_Hutton',
   '512px-Hans_Talhoffer',
   '512px-George_E._Leland',
   '512px-PachecoNarvaez',
   '512px-Nikolai_Panin',
   '512px-LTC_Herman_Koehler,_USMA_Master_of_the_Sword',
   '512px-ItaloSantelli1896',
   '512px-Laszlo_Fejes_Toth',
   "512px-Arte_dell'Armi_-_Achille_Marozzo",
   '512px-WAGC_DMT',
   '512px-Gheorghe_Moceanu',
   '512px-Gabriella_Papadakis_and_Guillaume_Cizeron_at_2018_Internationaux_de_France-Ice_dancing-IMG_6333',
   '512px-Bartitsu_montage',
   '512px-Erna_Juel-Hansen-adl',
   '512px-Corps_Berlin_320',
   '512px-Adolf_Spiess',
   '512px-Genevieve_Stebbins_Astley_1902',
   '512px-Colonel_Francis_Maceroni',
   '512px-Kleist,_Heinrich_von',
   '512px-Georges_Clemenceau_par_Nadar',
   '512px-ErnestBevinSchool',
   '512px-Бутовський_Олексій_Дмитрович',
   '512px-Foto_yrigoyen_-_presidente_-presidenciagovar',
   '512px-Bust_of_Gobar_Goho',
   

In [48]:
torch.manual_seed(seed+4)

In [42]:
save_name = osp.join(temp_dir, '{}_{}.pt'.format(ex.current_run.config['model']['name'],
                                                         ex.current_run.config['dataset']['name']))

AttributeError: 'NoneType' object has no attribute 'config'

In [52]:
result

{'map': 30.21,
 'mrr': 44.19,
 'precision': 12.98,
 'hit_rate': 82.0,
 'recall': 78.41,
 'map@1': 11.01,
 'mrr@1': 35.0,
 'precision@1': 35.0,
 'hit_rate@1': 35.0,
 'recall@1': 11.01,
 'map@5': 17.79,
 'mrr@5': 42.2,
 'precision@5': 19.6,
 'hit_rate@5': 55.0,
 'recall@5': 22.5,
 'map@6': 18.59,
 'mrr@6': 42.53,
 'precision@6': 18.5,
 'hit_rate@6': 57.0,
 'recall@6': 24.58,
 'map@10': 21.27,
 'mrr@10': 43.04,
 'precision@10': 16.2,
 'hit_rate@10': 61.0,
 'recall@10': 32.04}

In [51]:
for epoch in range(1):
    if cudnn_flag == 'benchmark':
        setattr(cudnn, cudnn_flag, True)

    torch.cuda.empty_cache()
    train_one_epoch(model=model, loader=loaders.train, class_loss=class_loss, optimizer=optimizer, scheduler=scheduler, max_norm=max_norm, epoch=epoch, freq=visdom_freq, ex=None)
    
    # validation
    if cudnn_flag == 'benchmark':
        setattr(cudnn, cudnn_flag, False)
    
    torch.cuda.empty_cache()
    result, query_feats, gallery_feats = fast_evaluate_viquae(model=model,
                                                              cache_nn_inds=cache_nn_inds,
                                                              recall=recall_ks,
                                                              query_loader=loaders.query, 
                                                              gallery_loader=loaders.gallery,
                                                              query_feats=query_feats, 
                                                              gallery_feats=gallery_feats)
    
    print('Validation [{:03d}]'.format(epoch)), pprint(result)
    #ex.log_scalar('val.map', result['map'], step=epoch + 1)

    if result['map'] >= best_val[1]['map']:
        print('New best model in epoch %d.'%epoch)
        best_val = (epoch + 1, result, deepcopy(model.state_dict()))
        #torch.save({'state': state_dict_to_cpu(best_val[2]), 'optim': optimizer.state_dict()}, save_name)

100%|██████████| 100/100 [01:02<00:00,  1.59it/s]


{'map': 30.21, 'mrr': 44.19, 'precision': 12.98, 'hit_rate': 82.0, 'recall': 78.41, 'map@1': 11.01, 'mrr@1': 35.0, 'precision@1': 35.0, 'hit_rate@1': 35.0, 'recall@1': 11.01, 'map@5': 17.79, 'mrr@5': 42.2, 'precision@5': 19.6, 'hit_rate@5': 55.0, 'recall@5': 22.5, 'map@6': 18.59, 'mrr@6': 42.53, 'precision@6': 18.5, 'hit_rate@6': 57.0, 'recall@6': 24.58, 'map@10': 21.27, 'mrr@10': 43.04, 'precision@10': 16.2, 'hit_rate@10': 61.0, 'recall@10': 32.04}
Validation [000]
{'hit_rate': 82.0,
 'hit_rate@1': 35.0,
 'hit_rate@10': 61.0,
 'hit_rate@5': 55.0,
 'hit_rate@6': 57.0,
 'map': 30.21,
 'map@1': 11.01,
 'map@10': 21.27,
 'map@5': 17.79,
 'map@6': 18.59,
 'mrr': 44.19,
 'mrr@1': 35.0,
 'mrr@10': 43.04,
 'mrr@5': 42.2,
 'mrr@6': 42.53,
 'precision': 12.98,
 'precision@1': 35.0,
 'precision@10': 16.2,
 'precision@5': 19.6,
 'precision@6': 18.5,
 'recall': 78.41,
 'recall@1': 11.01,
 'recall@10': 32.04,
 'recall@5': 22.5,
 'recall@6': 24.58}


TypeError: tuple indices must be integers or slices, not str

In [ ]:
gnd =  pickle_load(osp.join(train_data_dir, train_gnd_file))

In [ ]:
gnd['gnd'][0].keys()

In [ ]:
train_lines     = read_file(osp.join(train_data_dir, train_txt))
train_samples   = [(line.split(split_char)[0], int(line.split(split_char)[1]), int(line.split(split_char)[2]), int(line.split(split_char)[3])) for line in train_lines]
train_set       = FeatureDataset(train_data_dir, train_samples, desc_name, max_sequence_len)
query_train_set = FeatureDataset(train_data_dir, train_samples, desc_name, max_sequence_len)

In [ ]:
#s_name = set_name
#if s_name != '':
#    s_name = set_name + '_'
#def map_nnids_labels(train_data_dir, train_gnd_file, s_categories):
#    gnd =  pickle_load(osp.join(train_data_dir, train_gnd_file))
#    selection_gallery = gnd['simlist']
#    s_categories = s_categories.reshape(np.array(selection_gallery).shape)
#    selection_ids_to_cat_dict = [{k: s_categories[i][k] for k in range(len(selection_gallery[i]))} for i in range(len(selection_gallery))]
#    print(s_categories.shape)
#
#    return selection_ids_to_cat_dict
#
#s_path = train_data_dir+'/'+set_name+'_s_categories.txt'
#s_categories = np.loadtxt(s_path, dtype='int64')
#map_nnids_labels = map_nnids_labels(train_data_dir, train_gnd_file, s_categories)

train_nn_inds = osp.join(train_data_dir, set_name + '_nn_inds_%s.pkl'%desc_name)
train_sampler = TripletSampler(train_set.targets, batch_size, train_nn_inds, num_candidates, gnd['gnd'])

In [ ]:
#nn_inds_path =  osp.join(train_data_dir, 'training_' + s_name+'nn_inds_%s.pkl'%desc_name)
#cache_nn_inds = torch.from_numpy(pickle_load(nn_inds_path)).long()
#cache_nn_inds.shape

In [ ]:
len(train_sampler.valids), np.where(train_sampler.valids > 0)[0]

In [ ]:
for item in train_sampler:
    print(item)

In [ ]:
len(train_sampler)

In [ ]:
len(item)

In [ ]:
gnd['gnd'][10]['hard']

In [ ]:
gnd['simlist'][1][6:17]